# DataProcessing
Pipeline: raw Excel/CSV ➜ Knowledge Base sản phẩm + Transactions (bill) + Artifacts (Content TF-IDF + Rules).

In [ ]:
import os
import re
import json
import math
import ast
from pathlib import Path
from typing import List, Dict, Tuple, Optional

import numpy as np
import pandas as pd

# =========================
# In/Out Dir (QUẢN LÝ RÕ RÀNG)
# =========================
RAW_DIR   = Path("./Data")  # chứa file gốc: all_barcodes.xlsx, products_data.xlsx, categories_data.xlsx, brands_data.xlsx, countries_of_origin.xlsx
OUT_DIR   = Path("./DataAfterProcessing")
PROC_DIR  = OUT_DIR / "Processed"
STATS_DIR = OUT_DIR / "Stats"
MODELS_DIR = Path("./Models")  # model/rules
RULES_DIR  = MODELS_DIR / "Rules"
CONTENT_DIR= MODELS_DIR / "Content"

for d in [RAW_DIR, OUT_DIR, PROC_DIR, STATS_DIR, MODELS_DIR, RULES_DIR, CONTENT_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# =========================
# Input file names
# =========================
F_ALL_BARCODES = RAW_DIR / "all_barcodes.xlsx"
F_PRODUCTS     = RAW_DIR / "products_data.xlsx"
F_CATEGORIES   = RAW_DIR / "categories_data.xlsx"
F_BRANDS       = RAW_DIR / "brands_data.xlsx"
F_COUNTRIES    = RAW_DIR / "countries_of_origin.xlsx"

# =========================
# Output files
# =========================
F_PRODUCTS_ENRICHED = PROC_DIR / "products_enriched.csv"
F_TX_LONG          = PROC_DIR / "transactions_long.csv"
F_UNKNOWN_CODES    = PROC_DIR / "unknown_barcodes.csv"  # các mã trong bill không map được sang product

print("RAW_DIR:", RAW_DIR.resolve())
print("PROC_DIR:", PROC_DIR.resolve())
print("MODELS_DIR:", MODELS_DIR.resolve())


RAW_DIR: D:\TDTU\DA\DACNTT\Data
PROC_DIR: D:\TDTU\DA\DACNTT\DataAfterProcessing\Processed
MODELS_DIR: D:\TDTU\DA\DACNTT\Models


In [ ]:
def read_excel_safe(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"Không thấy file: {path.resolve()}")
    return pd.read_excel(path)

barcodes_raw  = read_excel_safe(F_ALL_BARCODES)
products_raw  = read_excel_safe(F_PRODUCTS)
cats_raw      = read_excel_safe(F_CATEGORIES)
brands_raw    = read_excel_safe(F_BRANDS)
countries_raw = read_excel_safe(F_COUNTRIES)

print("all_barcodes columns:", barcodes_raw.columns.tolist())
print("products columns    :", products_raw.columns.tolist())
print("categories columns  :", cats_raw.columns.tolist())
print("brands columns      :", brands_raw.columns.tolist())
print("countries columns   :", countries_raw.columns.tolist())

# Quick check
assert "product_codes" in barcodes_raw.columns, "all_barcodes.xlsx phải có cột product_codes"
required_products = ["_id","barcode","name","price","description","brand","country_of_origin","category"]
missing_p = [c for c in required_products if c not in products_raw.columns]
if missing_p:
    raise KeyError(f"products_data.xlsx thiếu cột: {missing_p}")

all_barcodes columns: ['product_codes']
products columns    : ['_id', 'barcode', 'name', 'price', 'description', 'brand', 'country_of_origin', 'category']
categories columns  : ['_id', 'name', 'slug', 'parent_category']
brands columns      : ['_id', 'name']
countries columns   : ['_id', 'name']


In [3]:
def clean_id(x) -> str:
    if pd.isna(x):
        return ""
    s = str(x).strip()
    # loại .0 khi excel đọc số
    s = re.sub(r"\.0$", "", s)
    return s

def clean_barcode(x) -> str:
    if pd.isna(x):
        return ""
    s = str(x).strip()
    s = re.sub(r"\.0$", "", s)
    s = s.replace(" ", "")
    return s

CODE_REGEX = re.compile(r"\d{6,}")  # barcode thường là chuỗi số dài

def parse_product_codes(cell) -> List[str]:
    """
    all_barcodes.xlsx: mỗi dòng = 1 bill, cột product_codes = 'a,b,c' (có thể dư dấu phẩy cuối)
    """
    if pd.isna(cell):
        return []
    s = str(cell).strip()
    if not s:
        return []
    # trường hợp cell là list string dạng "['..','..']"
    if s.startswith("[") and s.endswith("]"):
        try:
            arr = ast.literal_eval(s)
            if isinstance(arr, list):
                return [clean_barcode(x) for x in arr if str(x).strip()]
        except Exception:
            pass
    # mặc định: tách bằng regex numeric
    codes = CODE_REGEX.findall(s)
    return [clean_barcode(c) for c in codes if clean_barcode(c)]

# Test nhanh
print(barcodes_raw["product_codes"].head(2).tolist())
print(parse_product_codes(barcodes_raw["product_codes"].iloc[0])[:10])

['9300658408526,8935212813020,8850157400107,8934680012027,78895152869,8934868188650,9556123464720,8801047161608,8850199586586,8934673101080,8935212813495', '8935302600745,8859405999158,8934678010424,8850100004628,8936110890014,8935358700017,8936036026627,4902430451680,8935311110921,8936049053320,8934588130113,4987176320834,8936021360040,8935118500710,8934563106140,8934588243110,8936003724532,8934588672118,8801047161608,8936032987144']
['9300658408526', '8935212813020', '8850157400107', '8934680012027', '78895152869', '8934868188650', '9556123464720', '8801047161608', '8850199586586', '8934673101080']


In [ ]:
# Clean ids
products = products_raw.copy()
products["_id"] = products["_id"].apply(clean_id)
products["barcode"] = products["barcode"].apply(clean_barcode)
products["brand"] = products["brand"].apply(clean_id)
products["category"] = products["category"].apply(clean_id)
products["country_of_origin"] = products["country_of_origin"].apply(clean_id)
products["name"] = products["name"].astype(str).fillna("").str.strip()
products["description"] = products["description"].astype(str).fillna("").str.strip()

cats = cats_raw.copy()
cats["_id"] = cats["_id"].apply(clean_id)
cats["name"] = cats["name"].astype(str).fillna("").str.strip()
# file có slug, không bắt buộc dùng
if "parent_category" in cats.columns:
    cats["parent_category"] = cats["parent_category"].apply(clean_id)
else:
    cats["parent_category"] = ""

brands = brands_raw.copy()
brands["_id"] = brands["_id"].apply(clean_id)
brands["name"] = brands["name"].astype(str).fillna("").str.strip()

countries = countries_raw.copy()
countries["_id"] = countries["_id"].apply(clean_id)
countries["name"] = countries["name"].astype(str).fillna("").str.strip()

brand_map = brands.set_index("_id")["name"].to_dict()
country_map = countries.set_index("_id")["name"].to_dict()
cat_name_map = cats.set_index("_id")["name"].to_dict()
cat_parent_map = cats.set_index("_id")["parent_category"].to_dict()

def get_parent_chain(cat_id: str, max_hops=5) -> List[str]:
    """
    Trả về chain [ParentName, ChildName] (tối đa max_hops để tránh loop)
    """
    chain = []
    cur = cat_id
    hops = 0
    while cur and hops < max_hops:
        name = cat_name_map.get(cur, "")
        parent = cat_parent_map.get(cur, "")
        if name:
            chain.append(name)
        if not parent or parent == cur:
            break
        cur = parent
        hops += 1
    chain = chain[::-1]  # từ root -> leaf
    return chain

def get_parent_name(cat_id: str) -> str:
    p = cat_parent_map.get(cat_id, "")
    return cat_name_map.get(p, "") if p else ""

print("Số category:", len(cats), "| Số brand:", len(brands), "| Số country:", len(countries))

Số category: 389 | Số brand: 1430 | Số country: 41


In [ ]:
prod = products.copy()

prod["product_id_str"] = prod["_id"].astype(str)
prod["product_name_vi"] = prod["name"].astype(str)

prod["brand_name"] = prod["brand"].map(brand_map).fillna("UNKNOWN_BRAND")
prod["country_name"] = prod["country_of_origin"].map(country_map).fillna("UNKNOWN_COUNTRY")

prod["category_id"] = prod["category"].astype(str)
prod["category_name"] = prod["category_id"].map(cat_name_map).fillna("UNKNOWN_CATEGORY")
prod["parent_category_id"] = prod["category_id"].map(cat_parent_map).fillna("")
prod["parent_category_name"] = prod["parent_category_id"].map(cat_name_map).fillna("")

# Chuỗi category path: Parent > Child
def cat_path(cid: str) -> str:
    chain = get_parent_chain(cid, max_hops=6)
    chain = [c for c in chain if c]
    return " > ".join(chain) if chain else "UNKNOWN_CATEGORY"

prod["category_path"] = prod["category_id"].apply(cat_path)

# Rich text (Knowledge Base) cho semantic/content
# (Không remove dấu ở đây; giữ nguyên cho tiếng Việt)
def safe_join(parts: List[str]) -> str:
    out = []
    for p in parts:
        p = str(p).strip()
        if p and p not in ["nan","None"]:
            out.append(p)
    return " | ".join(out)

prod["full_metadata"] = prod.apply(
    lambda r: safe_join([
        r["product_name_vi"],
        r["brand_name"],
        r["category_path"],
        r["country_name"],
        r.get("description","")
    ]),
    axis=1
)

# Giá
prod["price"] = pd.to_numeric(prod["price"], errors="coerce").fillna(0).astype(int)

# Chỉ giữ cột cần thiết
products_enriched = prod[[
    "product_id_str","barcode","product_name_vi","price",
    "brand_name","country_name",
    "category_id","category_name",
    "parent_category_id","parent_category_name",
    "category_path",
    "description","full_metadata"
]].copy()

print(products_enriched.head(2))
products_enriched.to_csv(F_PRODUCTS_ENRICHED, index=False, encoding="utf-8")
print("Saved:", F_PRODUCTS_ENRICHED)

             product_id_str        barcode  \
0  68311cba419bc51ab2ee5aa3  8934868172239   
1  68311cba419bc51ab2ee5aa5  8934868152576   

                                     product_name_vi   price brand_name  \
0  Dầu Gội Nam Sạch Gàu Clear Men Hương Gió Biển ...  249000  Clear Men   
1  Dầu Gội Sạch Gàu Clear Bạc Hà Mát Lạnh Chai 1.4kg  370000      Clear   

      country_name               category_id    category_name  \
0  UNKNOWN_COUNTRY  682fdf11419bc50c9f24af11  Dầu Gội, Dầu Xả   
1  UNKNOWN_COUNTRY  682fdf11419bc50c9f24af11  Dầu Gội, Dầu Xả   

         parent_category_id parent_category_name  \
0  682fdf06419bc50c9f24aefb         Chăm Sóc Tóc   
1  682fdf06419bc50c9f24aefb         Chăm Sóc Tóc   

                                       category_path  \
0  Chăm Sóc Cá Nhân > Chăm Sóc Nam Giới > Chăm Só...   
1  Chăm Sóc Cá Nhân > Chăm Sóc Nam Giới > Chăm Só...   

                                         description  \
0  ƯU ĐIỂM NỔI BẬT:– SẠCH GÀU, HẾT NGỨA:Công nghệ...   
1

In [6]:
# all_barcodes: mỗi dòng là 1 bill -> tạo bill_id từ index
tx = barcodes_raw.copy()
tx["bill_id"] = np.arange(len(tx)).astype(int)

tx["codes"] = tx["product_codes"].apply(parse_product_codes)
tx_long = tx[["bill_id","codes"]].explode("codes").rename(columns={"codes":"barcode"})
tx_long["barcode"] = tx_long["barcode"].astype(str).str.strip()
tx_long = tx_long[tx_long["barcode"] != ""].dropna()

print("Số dòng (bill) ban đầu:", tx["bill_id"].nunique())
print("Sau explode, tổng số dòng barcode:", len(tx_long))

# Join với products_enriched theo barcode
barcode_to_pid = products_enriched.set_index("barcode")["product_id_str"].to_dict()

tx_long["product_id_str"] = tx_long["barcode"].map(barcode_to_pid).fillna("")
coverage = (tx_long["product_id_str"] != "").mean()

print(f"Join coverage (barcode->product): {coverage:.3f}")

unknown = tx_long[tx_long["product_id_str"] == ""][["bill_id","barcode"]].copy()
unknown.to_csv(F_UNKNOWN_CODES, index=False, encoding="utf-8")

# Giữ dòng map được
transactions_long = tx_long[tx_long["product_id_str"] != ""].copy()

# Merge thêm thông tin sản phẩm cho EDA / rules
transactions_long = transactions_long.merge(
    products_enriched[[
        "product_id_str","product_name_vi","price",
        "brand_name","country_name",
        "category_id","category_name",
        "parent_category_name","category_path"
    ]].drop_duplicates("product_id_str"),
    on="product_id_str",
    how="left"
)

print("transactions_long shape:", transactions_long.shape)
transactions_long.head()

Số dòng (bill) ban đầu: 4430
Sau explode, tổng số dòng barcode: 49799
Join coverage (barcode->product): 1.000
transactions_long shape: (49786, 11)


,bill_id,barcode,product_id_str,product_name_vi,price,brand_name,country_name,category_id,category_name,parent_category_name,category_path
0,0,9300658408526,683129de419bc51ab2ee7179,Sữa Chua Ăn Kiểu Hy Lạp Farmers Union Vị Vani ...,20900,FARMERS UNION,Australia,682fdceb419bc5d02ce660e4,Sữa Chua Ăn,"Sữa Chua, Váng Sữa","Chế Phẩm Từ Sữa > Sữa > Sữa Chua, Váng Sữa > S..."
1,0,8935212813020,68312d8e419bc51ab2ee7760,Sữa Tắm Romano Classic Sạch Khuẩn 650g,185900,Wipro,UNKNOWN_COUNTRY,682fdf11419bc50c9f24af0f,"Tắm, Gội",Làm Sạch Cơ Thể,Chăm Sóc Cá Nhân > Chăm Sóc Nam Giới > Làm Sạc...
2,0,8850157400107,68312f1a419bc51ab2ee79e3,Snack Mực Tẩm Gia Vị Cay Ngọt Bento 18g,28500,UNKNOWN_BRAND,UNKNOWN_COUNTRY,682fde17419bc5e8a44ee68b,Bánh Snack,"Snack, Ăn Vặt","Bánh Kẹo > Snack, Ăn Vặt > Bánh Snack"
3,0,8934680012027,68312f73419bc51ab2ee7a64,Khoai Tây Chiên Slide Vị Thơm Cay 90G,32600,Slide,UNKNOWN_COUNTRY,682fde17419bc5e8a44ee68b,Bánh Snack,"Snack, Ăn Vặt","Bánh Kẹo > Snack, Ăn Vặt > Bánh Snack"
4,0,78895152869,68312617419bc51ab2ee6b33,Nước Tương Đậu Nành Lee Kum Kee Chai 500ml,22500,LEE KUM KEE,Malaysia,682fdc7f419bc5bc406ac822,Gia Vị Hoàn Chỉnh,"Sốt, Gia Vị Các Loại","Dầu Ăn, Gia Vị > Gia Vị > Sốt, Gia Vị Các Loại..."


In [7]:
# Popularity: số lần product xuất hiện trong bill
prod_pop = (
    transactions_long.groupby("product_id_str")["bill_id"]
    .count()
    .rename("n_appear")
    .reset_index()
)

products_enriched2 = products_enriched.merge(prod_pop, on="product_id_str", how="left")
products_enriched2["n_appear"] = products_enriched2["n_appear"].fillna(0).astype(int)

# Chuẩn hóa popularity để dùng rank
mx = max(products_enriched2["n_appear"].max(), 1)
products_enriched2["popularity_norm"] = products_enriched2["n_appear"] / mx

# Save lại (enriched + popularity)
products_enriched2.to_csv(F_PRODUCTS_ENRICHED, index=False, encoding="utf-8")
transactions_long.to_csv(F_TX_LONG, index=False, encoding="utf-8")

print("Saved:", F_PRODUCTS_ENRICHED)
print("Saved:", F_TX_LONG)
print("Unknown barcodes saved:", F_UNKNOWN_CODES, "| n_unknown:", len(unknown))

Saved: DataAfterProcessing\Processed\products_enriched.csv
Saved: DataAfterProcessing\Processed\transactions_long.csv
Unknown barcodes saved: DataAfterProcessing\Processed\unknown_barcodes.csv | n_unknown: 13
